# Multiple Object Tracking Accuracy

1. Convert a set of images, which are labeled for YOLO training, into MOTA format
2. Use these images as a video for evaluating

    Annoyingly, mtometrics and the Yolo-Deepsort code have conflicting dependencies, so a new virtual environment with numpy<2 is necesary. 
      
    **Alternatively**, we can just edit lines 117 and 118 in the `distances.py` module from 

        objs = np.asfarray(objs)
        hyps = np.asfarray(hyps)
    
    to 

        objs = np.asarray(objs)
        hyps = np.asarray(hyps)
    


## 1. Imports and initializing tracker

In [ ]:
from mota_utils import (calculate_iou, centroid_distances, convert_yolo_to_mota, 
                        display_mota_labels, evaluate_tracker, disp_compare_gt_pred)
from detection_helpers import Detector
# from tracking_helpers import *
from  bridge_wrapper import YoloDeepsortUtility, TRACKER_PARAMS

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
detector = Detector(classes = [0,1], conf_thres=TRACKER_PARAMS["conf_thres"], iou_thresh=TRACKER_PARAMS["iou_thresh"]) # class = None means detect all classes. List info at: "data/coco.yaml"
detector.load_model(TRACKER_PARAMS["model_path"],img_size=1088) # pass the path to the trained weight file
tracker = YoloDeepsortUtility(detector=detector,params_dict=TRACKER_PARAMS)
# tracker = JustYoloUtility(detector) #Object detection without tracking
tracker.satturation = 2 # satturation increase. doesn't seem to affect much



d:\Proyectos\HAU\codigo\3-path-planning\yolov7_deepsort_tracking\models\experimental.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(w, map_location=m

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

nn_budget not in params_dict. Using default


## 2. Track and compare
MOTChallenge metrics can be obtained with the `evaluate_tracker` function.

Predicted and ground truth bounding boxes can be displayed with `disp_compare_gt_pred` 

`main_compare`

In [6]:
img_index="1b"
tracker.track_video_from_image_folder(img_folder=f"vid-seg1", output=f"pred_labels1cust-ds.txt")
ev = evaluate_tracker(f"mota_labels{img_index}.txt", f"pred_labels1cust-ds.txt", ignore_id=None, do_print=False, maxiou=0.9) #ignore_id=None does best
ev

Tracking frame 0/74
Tracking frame 1/74
Tracking frame 2/74
Tracking frame 3/74
Tracking frame 4/74
Tracking frame 5/74
Tracking frame 6/74
Tracking frame 7/74
Tracking frame 8/74
Tracking frame 9/74
Tracking frame 10/74
Tracking frame 11/74
Tracking frame 12/74
Tracking frame 13/74
Tracking frame 14/74
Tracking frame 15/74
Tracking frame 16/74
Tracking frame 17/74
Tracking frame 18/74
Tracking frame 19/74
Tracking frame 20/74
Tracking frame 21/74
Tracking frame 22/74
Tracking frame 23/74
Tracking frame 24/74
Tracking frame 25/74
Tracking frame 26/74
Tracking frame 27/74
Tracking frame 28/74
Tracking frame 29/74
Tracking frame 30/74
Tracking frame 31/74
Tracking frame 32/74
Tracking frame 33/74
Tracking frame 34/74
Tracking frame 35/74
Tracking frame 36/74
Tracking frame 37/74
Tracking frame 38/74
Tracking frame 39/74
Tracking frame 40/74
Tracking frame 41/74
Tracking frame 42/74
Tracking frame 43/74
Tracking frame 44/74
Tracking frame 45/74
Tracking frame 46/74
Tracking frame 47/74
Tr

,idf1,idp,idr,recall,precision,...,mota,motp,num_transfer,num_ascend,num_migrate
overall,0.828862,0.828096,0.825046,0.915285,0.923792,...,0.823204,0.405187,5,4,4


In [196]:
disp_compare_gt_pred(image_folder="vid-seg1", gt_labels_file="mota_labels1b.txt", pred_labels_file="pred_labels1b.txt")

Frame 000030 not found.
Frame 000031 not found.


In [ ]:

def main_compare(tracker, img_index, track=True, iou_threshold=10, cent_threshold=0.05, stop_at_frame=None, ignore_id=[0]):
    """ Not recomended, but can perform the whole process of converting YOLO labels to MOTA labels, tracking, and obtaining metrics"""
    
    convert_yolo_to_mota(
    image_folder=f"vid-seg{img_index}",
    label_folder=f"vid-seg{img_index}",
    output_file=f"mota_labels{img_index}.txt",
    iou_threshold=iou_threshold,
    cent_threshold=cent_threshold,
    stop_at_frame=stop_at_frame)
    
    if track:
        tracker.track_video_from_image_folder(img_folder=f"vid-seg{img_index}", output=f"pred_labels{img_index}.txt")

    return evaluate_tracker(f"mota_labels{img_index}.txt", f"pred_labels{img_index}.txt", ignore_id=ignore_id, do_print=False) # ignoring crops (id 0)

In [ ]:
summary = main_compare(tracker, track=False, img_index=1, iou_threshold=10, cent_threshold=0.05, ignore_id=[0], stop_at_frame=None)
summary
#weird. QUalitatively, this looks much worse than cent=0.05

,mota,motp
overall,0.197417,0.329534


In [ ]:

convert_yolo_to_mota(
    image_folder="vid-seg1",
    label_folder="vid-seg1",
    output_file="mota_labels.txt",
    iou_threshold=10,
    cent_threshold=0.05,
    stop_at_frame=None
)


             mota     motp
overall  0.326172  0.31707


In [93]:
display_mota_labels(
    image_folder="vid-seg1",
    mota_labels_file="mota_labels1.txt"
)

In [217]:
display_mota_labels(
    image_folder="vid-seg1",
    mota_labels_file="pred_labels1.txt"
)